## **REPORTE KPIS COMERCIAL**

#### **Librerías**

In [2]:
import os
import math
import time
import smtplib

import numpy as np
import pandas as pd

import xlsxwriter

from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from google.cloud import bigquery

#### **Funciones**

In [3]:
# Run BigQuery query and convert to dataframe
def run_bigquery_query(query):
    # Project and client
    project_id = 'bi-fcom-drmb-local-pe-sbx'
    client = bigquery.Client(project=project_id)

    # Job configuration
    job_config = bigquery.QueryJobConfig()
    job_config.query_parameters = (bigquery.ScalarQueryParameter("limit", "INT64", 1000),)

    # Query execution
    query_job = client.query(query, job_config=job_config)
    query_job.result()
    
    # Convert query result to dataframe
    data = query_job.to_dataframe()
    
    return data

#### **Queries**

In [4]:
# KPIs operativos (OTS + Cancels + Devoluciones)



# Devoluciones

query_devoluciones = """
    SELECT * FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs.reporte_devoluciones`
"""

# Tickets

query_tickets = """
    SELECT
        * 
        EXCEPT(
            country,commerce,
            week,
            caseStakeholder,caseOwner,caseBU,
            caseCreatedDate,caseDueDate,caseClosedDate,
            orders,skus,
            groupcaseTipification,Departamento,Area,
            Origin,caseSLA,caseSLAhours,
            caseReopenings,caseRecontacts,
            regulatoryId,refund_id,refund_amount,refund_status,refund_date
            case_due_created_hours,case_completion_hours,case_diff_completion_due_hours,
            case_due_created_work_hours,case_completion_work_hours,case_diff_completion_work_hours,
            late,row,Sector,cumplimientoSLA
        )
    FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs.reporte_casos_salesforce`
    WHERE caseLevel1 IN ('Reclamo','Pedido Solicitud')
    AND sellerName IS NOT NULL
    AND LENGTH(sellerName)!=0
"""

# Reviews

query_reviews = """
    SELECT * FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs.reporte_customer_reviews`
"""

# Apelaciones

query_apelaciones = """
    SELECT * FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs.seguimiento_apelaciones`
"""

#### **Dataframes**

In [ ]:
data_devoluciones = run_bigquery_query(query_devoluciones)
data_tickets = run_bigquery_query(query_tickets) # 7min
data_reviews = run_bigquery_query(query_reviews)
data_apelaciones = run_bigquery_query(query_apelaciones)

#### **Vistas**

In [5]:
data_tickets = run_bigquery_query(query_tickets)

c:\Users\gfloress\AppData\Local\Programs\Python\Python39\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [12]:
(
    data_tickets[(data_tickets['sellerId'] == 'SCD1016') & (data_tickets['caseLevel1']=='Reclamo')]
    .sort_values(['date'], ascending=False)
    .drop(['caseCreatedDate','caseDueDate','caseClosedDate',
           'regulatoryId','refund_id','refund_amount','refund_status','refund_date'], axis=1)
)

,year,month,date,customerDocument,customerName,sellerId,sellerName,orderNumber,deliveryOrderNumber,sku,...,caseDescription,casePriority,caseCreatedDate,caseDueDate,caseClosedDate,caseClosureType,caseClosureComment,value,segmento,tipoReclamo
152607,2024,4,2024-04-27,48299436,Mary Gabriela Uchasara Gonzales,SCD1016,BATA PERU,1196134079,2196083582,115339522,...,https appsusw2pureclouddirectory engageadminin...,Medium,2024-04-27 10:31:14,2024-05-04 13:31:00,2024-04-27 14:19:26,Procede,cierre orden lista para retiro se informa via ...,128.000000000,AAA,Problemas Entrega
203903,2024,4,2024-04-27,42200304,Esther Gomez,SCD1016,BATA PERU,1196918292,2197738272,127875423,...,sin comentarios por el cliente motivo seller d...,Medium,2024-04-27 09:10:05,2024-05-04 12:10:00,2024-04-27 16:21:35,Procede,989045416 no responde llamadas genesys f9 list...,97.930000000,AAA,Problemas Entrega
162271,2024,4,2024-04-27,41661279,Eder Walter Celis,SCD1016,BATA PERU,1195663420,2195448906,124093949,...,2195448906 cliente se comunica para cancelar c...,Medium,2024-04-27 17:36:43,2024-05-06 12:00:00,NaT,None,None,83.930000000,AAA,Problemas Entrega
3075,2024,4,2024-04-26,40478235,Leslie Alayo,SCD1016,BATA PERU,1196441731,2196692871,124062754,...,viene del caso 32026648 cliente desea cambio d...,Medium,2024-04-26 16:37:50,2024-05-16 13:37:00,NaT,None,None,67.960000000,AAA,Seller
149249,2024,4,2024-04-26,75869744,Thalia Gonzales Romualdo,SCD1016,BATA PERU,1194689533,2193739053,114452425,...,id fbf1b610 36b8 4a44 91c1 6c12babda535 cl se ...,Medium,2024-04-26 17:14:50,2024-05-03 20:14:00,2024-04-26 21:45:56,Procede,cierre se informa que compra se encuentra en p...,118.930000000,AAA,Problemas Entrega
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181372,2022,10,2022-10-26,07068000,Ayde Luz Nanez,SCD1016,BATA PERU,1086100132,2034083223,114436237,...,solicitud de cancelacion,Medium,2022-10-26 22:47:03,2022-11-03 12:00:00,2022-10-26 22:57:04,Procede,cierre por evento de pago,103.920000000,AAA,Otro
181901,2022,10,2022-10-26,07245102,Walda Del Carmen Silva,SCD1016,BATA PERU,1086108118,2034087948,114464478,...,solicitud de cancelacion,Medium,2022-10-26 22:47:19,2022-11-03 12:00:00,2022-10-26 22:57:21,Procede,cierre por evento de pago,87.920000000,AAA,Otro
181395,2022,10,2022-10-22,09112596,Maria Elena Tinoco,SCD1016,BATA PERU,1085332528,2033604784,114436732,...,solicitud de cancelacion,Medium,2022-10-22 10:21:25,2022-10-28 13:51:00,2022-10-22 10:31:28,Procede,cierre por evento de pago,179.900000000,AAA,Otro
27263,2022,10,2022-10-22,48804394,Angi Malca,SCD1016,BATA PERU,1085745390,2033873797,114465686,...,solicitar guia de conformidad no me fue entreg...,Medium,2022-10-22 15:44:46,2022-11-11 13:14:00,2022-12-16 11:44:01,No procede,se cierra ticket por falta de datos si cliente...,199.800000000,AAA,Problemas Entrega


#### **Exportar info**

##### **1. Reportes semanales para los KAMs**

In [39]:
if reporte_semanal_sellers!='yes':

    ## Ruta
    final_path = ruta_pagos +  r'\\0. Reportes\\2. Reportes CSV\\' + str(pd.to_datetime(fecha_inicio, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','') + '-' + str(pd.to_datetime(fecha_fin, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','')

    ## Listado de tablas a exportar
    lista = [resultado_ventas, resultado_egresos]
    hojas = ['BASE FACTURA', 'BASE ABONO']

    ## Nombre del archivo Excel
    writer = pd.ExcelWriter(os.path.join(final_path, ('Reporte de pagos y descuentos.xlsx')), engine='xlsxwriter')
            
    ## Exportar Excel
    for j, df in enumerate(lista):
            ## Excel
            df.to_excel(writer, sheet_name=hojas[j], startrow=1, index=False, header=False)
            ## Define workbook and worksheet
            workbook = writer.book
            worksheet = writer.sheets[hojas[j]]
            ## Header format
            header_format = workbook.add_format({'bold': True, 'text_wrap': False, 'valign': 'top', 'fg_color': '#D7E4BC'})
            ## Write headers with the defined format
            for col_num, value in enumerate(df.columns.values):
                worksheet.write(0, col_num, value, header_format)
            ## Format Workbook
            workbook.formats[0].set_font_size(10) # Tamaño 10 letra
            ## Format Worksheet
            worksheet.set_zoom(70) # 70% zoom
            worksheet.freeze_panes(1, 0) # Freeze row 1
            worksheet.hide_gridlines(2) # Hide gridlines

            ## Widths

            writer.sheets[hojas[j]].set_column('A:A', 20)
            writer.sheets[hojas[j]].set_column('B:B', 31, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('C:C', 19, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('D:D', 13)
            writer.sheets[hojas[j]].set_column('E:E', 45)
            writer.sheets[hojas[j]].set_column('F:F', 29, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('G:G', 11)
            writer.sheets[hojas[j]].set_column('H:H', 20, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('I:I', 18)
            writer.sheets[hojas[j]].set_column('J:J', 8, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('L:L', 41, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('M:M', 16)
            writer.sheets[hojas[j]].set_column('N:N', 15)
            writer.sheets[hojas[j]].set_column('O:O', 60, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('P:P', 12)
            writer.sheets[hojas[j]].set_column('Q:Q', 18)
            writer.sheets[hojas[j]].set_column('R:R', 15)
            writer.sheets[hojas[j]].set_column('S:S', 10)
            writer.sheets[hojas[j]].set_column('T:T', 14)
            writer.sheets[hojas[j]].set_column('U:U', 15)
            writer.sheets[hojas[j]].set_column('V:V', 12)
            writer.sheets[hojas[j]].set_column('W:W', 15)
            writer.sheets[hojas[j]].set_column('X:X', 9)
            writer.sheets[hojas[j]].set_column('Y:Y', 9)

    ## Filtros

    writer.sheets[hojas[0]].autofilter(0, 0, resultado_ventas.shape[0], resultado_ventas.shape[1]-1) 
    writer.sheets[hojas[1]].autofilter(0, 0, resultado_egresos.shape[0], resultado_egresos.shape[1]-1) 

    ## Guardar Excel
    writer.save()

else:
    
    print("Solo se procesará los reportes para sellers")

In [40]:
if reporte_semanal_sellers!='yes':

    kams_lista = kams_contactos['Nombre  Completo'].astype(str).drop_duplicates().to_list()

    ## Ruta
    final_path = ruta_pagos +  r'\\0. Reportes\\2. Reportes CSV\\' + str(pd.to_datetime(fecha_inicio, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','') + '-' + str(pd.to_datetime(fecha_fin, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','') + r'\\Reportes por KAM'

    for i in range(len(kams_lista)):

        # 1. Definir

        kam = kams_contactos[kams_contactos['Nombre  Completo']==kams_lista[i]]['Nombre  Completo'].drop_duplicates().values[0]
        rep_ventas_kam = resultado_ventas.query("KAM.notnull()")
        rep_ventas_kam = rep_ventas_kam[rep_ventas_kam['KAM'].str.contains(kams_lista[i])].reset_index(drop=True)
        rep_ventas_kam = rep_ventas_kam[rep_ventas_kam['Fecha de Compra']>=fecha_inicio]

        lista = [rep_ventas_kam]
        hojas = ['BASE FACTURA']

        writer = pd.ExcelWriter(os.path.join(final_path, ('Reporte de liquidaciones') + ' - ' + str(kams_lista[i]) + '.xlsx'), engine='xlsxwriter')
            
        ## Exportar Excel
        for j, df in enumerate(lista):
            ## Excel
            df.to_excel(writer, sheet_name=hojas[j], startrow=1, index=False, header=False)
            ## Define workbook and worksheet
            workbook = writer.book
            worksheet = writer.sheets[hojas[j]]
            ## Header format
            header_format = workbook.add_format({'bold': True, 'text_wrap': False, 'valign': 'top', 'fg_color': '#D7E4BC'})
            ## Write headers with the defined format
            for col_num, value in enumerate(df.columns.values):
                worksheet.write(0, col_num, value, header_format)
            ## Format Workbook
            workbook.formats[0].set_font_size(10) # Tamaño 10 letra
            ## Format Worksheet
            worksheet.set_zoom(70) # 70% zoom
            worksheet.freeze_panes(1, 0) # Freeze row 1
            worksheet.hide_gridlines(2) # Hide gridlines

            ## Widths

            writer.sheets[hojas[j]].set_column('A:A', 20)
            writer.sheets[hojas[j]].set_column('B:B', 31, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('C:C', 19, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('D:D', 13)
            writer.sheets[hojas[j]].set_column('E:E', 45)
            writer.sheets[hojas[j]].set_column('F:F', 29, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('G:G', 11)
            writer.sheets[hojas[j]].set_column('H:H', 20, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('I:I', 18)
            writer.sheets[hojas[j]].set_column('J:J', 8, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('L:L', 41, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('M:M', 16)
            writer.sheets[hojas[j]].set_column('N:N', 15)
            writer.sheets[hojas[j]].set_column('O:O', 60, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('P:P', 12)
            writer.sheets[hojas[j]].set_column('Q:Q', 18)
            writer.sheets[hojas[j]].set_column('R:R', 15)
            writer.sheets[hojas[j]].set_column('S:S', 10)
            writer.sheets[hojas[j]].set_column('T:T', 14)
            writer.sheets[hojas[j]].set_column('U:U', 15)
            writer.sheets[hojas[j]].set_column('V:V', 12)
            writer.sheets[hojas[j]].set_column('W:W', 15)
            writer.sheets[hojas[j]].set_column('X:X', 9)
            writer.sheets[hojas[j]].set_column('Y:Y', 9)

            ## Filtros

            writer.sheets[hojas[j]].autofilter(0, 0, rep_ventas_kam.shape[0], rep_ventas_kam.shape[1]-1)

        ## Guardar Excel
        writer.save()

else:
    
    print("Solo se procesará los reportes para sellers")

##### **2. Templates de recaudos y liquidaciones semanales**

In [41]:
if reporte_semanal_sellers!='yes':

    ## Ruta
    final_path = ruta_pagos +  r'\\0. Reportes\\2. Reportes CSV\\' + str(pd.to_datetime(fecha_inicio, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','') + '-' + str(pd.to_datetime(fecha_fin, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','')

    ## Listado de tablas a exportar
    lista = [rec_fcom, total_liq_reg_fcom]
    hojas = ['Recaudos', 'Liquidación']

    ## Nombre del archivo Excel
    writer = pd.ExcelWriter(os.path.join(final_path, ('Templates Recaudos + Liquidación.xlsx')), engine='xlsxwriter')
        
    ## Exportar Excel
    for j, df in enumerate(lista):
        ## Excel
        df.to_excel(writer, sheet_name=hojas[j], startrow=1, index=False, header=False)
        ## Define workbook and worksheet
        workbook = writer.book
        worksheet = writer.sheets[hojas[j]]
        ## Header format
        header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top', 'fg_color': '#D7E4BC'})
        ## Write headers with the defined format
        for col_num, value in enumerate(df.columns.values):
            worksheet.write(0, col_num, value, header_format)
        ## Format Workbook
        workbook.formats[0].set_font_size(10) # Tamaño 10 letra
        ## Format Worksheet
        worksheet.set_zoom(70) # 70% zoom
        worksheet.freeze_panes(1, 0) # Freeze row 1
        worksheet.hide_gridlines(2) # Hide gridlines

    ## Widths

    ### 1. Recaudos

    writer.sheets[hojas[0]].set_column('A:A', 23)
    writer.sheets[hojas[0]].set_column('B:B', 20)
    writer.sheets[hojas[0]].set_column('C:C', 26)
    writer.sheets[hojas[0]].set_column('D:D', 10)
    writer.sheets[hojas[0]].set_column('E:E', 9)
    writer.sheets[hojas[0]].set_column('F:F', 14)
    writer.sheets[hojas[0]].set_column('G:G', 13)
    writer.sheets[hojas[0]].set_column('H:H', 12)
    writer.sheets[hojas[0]].set_column('I:I', 14)
    writer.sheets[hojas[0]].set_column('J:J', 16)
    writer.sheets[hojas[0]].set_column('K:K', 22)
    writer.sheets[hojas[0]].set_column('L:L', 16)
    writer.sheets[hojas[0]].set_column('M:M', 12)
    writer.sheets[hojas[0]].set_column('N:N', 50)
    writer.sheets[hojas[0]].set_column('O:O', 13)
    writer.sheets[hojas[0]].set_column('P:P', 20)
    writer.sheets[hojas[0]].set_column('Q:Q', 20)
    writer.sheets[hojas[0]].set_column('R:R', 18)

    ### 2. Liquidación

    writer.sheets[hojas[1]].set_column('A:A', 23)
    writer.sheets[hojas[1]].set_column('B:B', 20)
    writer.sheets[hojas[1]].set_column('C:C', 26)
    writer.sheets[hojas[1]].set_column('D:D', 10)
    writer.sheets[hojas[1]].set_column('E:E', 9)
    writer.sheets[hojas[1]].set_column('F:F', 16)
    writer.sheets[hojas[1]].set_column('G:G', 12)
    writer.sheets[hojas[1]].set_column('H:H', 50)
    writer.sheets[hojas[1]].set_column('I:I', 22)
    writer.sheets[hojas[1]].set_column('J:J', 12)
    writer.sheets[hojas[1]].set_column('K:K', 13)
    writer.sheets[hojas[1]].set_column('L:L', 12)
    writer.sheets[hojas[1]].set_column('M:M', 14)
    writer.sheets[hojas[1]].set_column('N:N', 15)
    writer.sheets[hojas[1]].set_column('O:O', 20)
    writer.sheets[hojas[1]].set_column('P:P', 18)
    writer.sheets[hojas[1]].set_column('Q:Q', 15)
    writer.sheets[hojas[1]].set_column('R:R', 13)
    writer.sheets[hojas[1]].set_column('S:S', 20)
    writer.sheets[hojas[1]].set_column('T:T', 20)
    writer.sheets[hojas[1]].set_column('U:U', 18)

    ## Filtros

    writer.sheets[hojas[0]].autofilter(0, 0, rec_fcom.shape[0], rec_fcom.shape[1]-1) 
    writer.sheets[hojas[1]].autofilter(0, 0, total_liq_reg_fcom.shape[0], total_liq_reg_fcom.shape[1]-1) 

    ## Guardar Excel
    writer.save()

##### **3. Cruce SRX vs Recaudos F.com**

In [42]:
if reporte_semanal_sellers != 'yes':

    ## Ruta
    final_path = ruta_pagos +  r'\\0. Reportes\\2. Reportes CSV\\' + str(pd.to_datetime(fecha_inicio, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','') + '-' + str(pd.to_datetime(fecha_fin, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','')

    ## Listado de tablas a exportar
    lista = [validacion]
    hojas = ['data']

    ## Nombre del archivo Excel
    writer = pd.ExcelWriter(os.path.join(final_path, ('Recaudos SRX - F.com.xlsx')), engine='xlsxwriter')
        
    ## Exportar Excel
    for j, df in enumerate(lista):
        ## Excel
        df.to_excel(writer, sheet_name=hojas[j], index=False)
        ## Define workbook and worksheet
        workbook = writer.book
        worksheet = writer.sheets[hojas[j]]
        ## Format Workbook
        workbook.formats[0].set_font_size(10) # Tamaño 10 letra
        ## Format Worksheet
        worksheet.set_zoom(70) # 70% zoom
        worksheet.freeze_panes(1, 0) # Freeze row 1
        worksheet.hide_gridlines(2) # Hide gridlines

    ## Guardar Excel
    writer.save()

##### **4. Reporte consolidado de liquidaciones Falabella.com para team Pagos**

In [47]:
if reporte_semanal_sellers != 'yes':

    final_path = ruta_pagos + r'\\0. Reportes' 
    
     ## CalcFecha de nicio
    fecha_reporte=pd.to_datetime('2022-07-11')

    ## Calcular semana máxima reporte
    max_week = facturas_seller['Semana Recaudación'].max()

    ## Calcular fecha de compra máxima
    max_fecha_compra = rec_seller.query("`Fecha de Compra`.notnull()").loc[:, ['Semana Recaudación', 'Fecha de Compra']].drop_duplicates().groupby(['Semana Recaudación'], dropna=False).agg({'Fecha de Compra': 'max'}).reset_index().query("`Semana Recaudación` == @max_week")['Fecha de Compra'].max()

    ## Filtros
    rep_facturas_seller = facturas_seller
    rep_rec_seller = rec_seller.query("`Semana Recaudación` <= @max_week & `Fecha de Compra` >= @fecha_reporte")
    rep_liq_seller = liq_seller.query("`Semana Recaudación` <= @max_week & `Fecha de Compra` >= @fecha_reporte")
    rep_rec_sin_seller = rec_sin_seller.query("`Semana Recaudación` <= @max_week")
    rep_ajust_seller = ajust_seller.query("`Semana Recaudación` <= @max_week")
    rep_devoluciones_seller = devoluciones_seller.copy() #query("`Fecha de Compra` >= @fecha_coe & `Fecha de Compra` <= @max_fecha_compra")

    ## Listado de tablas a exportar
    lista = [rep_facturas_seller, rep_rec_seller, rep_liq_seller, rep_rec_sin_seller, rep_ajust_seller, rep_devoluciones_seller]
    hojas = ['Facturas', 'Recaudos', 'Liquidaciones', 'Rechazos', 'Ajustes', 'Devoluciones']
    ## Nombre del archivo Excel
    try:
        writer = pd.ExcelWriter(os.path.join(final_path, 'Reporte general de pagos y devoluciones - Falabella.com' + '.xlsx'), engine='xlsxwriter')
    except:
        writer = pd.ExcelWriter(os.path.join(final_path, 'Reporte general de pagos y devoluciones - Falabella.com' + str(pd.to_datetime('today').date()) + '.xlsx'), engine='xlsxwriter')
        
    ## Exportar Excel
    for j, df in enumerate(lista):
        ## Excel
        df.to_excel(writer, sheet_name=hojas[j], startrow=1, index=False, header=False)
        ## Define workbook and worksheet
        workbook = writer.book
        worksheet = writer.sheets[hojas[j]]
        ## Header format
        header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top', 'fg_color': '#D7E4BC'})
        ## Write headers with the defined format
        for col_num, value in enumerate(df.columns.values):
            worksheet.write(0, col_num, value, header_format)
        ## Format Workbook
        workbook.formats[0].set_font_size(10) # Tamaño 10 letra
        ## Format Worksheet
        worksheet.set_zoom(70) # 70% zoom
        worksheet.freeze_panes(1, 0) # Freeze row 1
        worksheet.hide_gridlines(2) # Hide gridlines

    ## Widths

    ### 1. Facturas

    writer.sheets[hojas[0]].set_column('A:A', 23)
    writer.sheets[hojas[0]].set_column('B:B', 16)
    writer.sheets[hojas[0]].set_column('C:C', 13)
    writer.sheets[hojas[0]].set_column('D:D', 55)
    writer.sheets[hojas[0]].set_column('E:E', 14)
    writer.sheets[hojas[0]].set_column('F:F', 25, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('G:G', 36, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('H:H', 17)
    writer.sheets[hojas[0]].set_column('I:I', 11)
    writer.sheets[hojas[0]].set_column('J:J', 14, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('K:K', 12, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('L:L', 14, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('M:M', 16, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('N:N', 17, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('O:O', 55)

    ### 2. Recaudos

    writer.sheets[hojas[1]].set_column('A:A', 23)
    writer.sheets[hojas[1]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('E:E', 12)
    writer.sheets[hojas[1]].set_column('F:F', 20)
    writer.sheets[hojas[1]].set_column('G:G', 18)
    writer.sheets[hojas[1]].set_column('H:H', 15)
    writer.sheets[hojas[1]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('M:M', 24)
    writer.sheets[hojas[1]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('Q:Q', 60)
    writer.sheets[hojas[1]].set_column('R:R', 12)
    writer.sheets[hojas[1]].set_column('S:S', 18)
    writer.sheets[hojas[1]].set_column('T:T', 15)
    writer.sheets[hojas[1]].set_column('U:U', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('V:V', 18, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('W:W', 8)

    ### 3. Liquidaciones

    writer.sheets[hojas[2]].set_column('A:A', 23)
    writer.sheets[hojas[2]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('E:E', 12)
    writer.sheets[hojas[2]].set_column('F:F', 20)
    writer.sheets[hojas[2]].set_column('G:G', 18)
    writer.sheets[hojas[2]].set_column('H:H', 15)
    writer.sheets[hojas[2]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('M:M', 24)
    writer.sheets[hojas[2]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('Q:Q', 60)
    writer.sheets[hojas[2]].set_column('R:R', 12)
    writer.sheets[hojas[2]].set_column('S:S', 18)
    writer.sheets[hojas[2]].set_column('T:T', 15)
    writer.sheets[hojas[2]].set_column('U:U', 15)
    writer.sheets[hojas[2]].set_column('V:V', 20)
    writer.sheets[hojas[2]].set_column('W:W', 16)
    writer.sheets[hojas[2]].set_column('X:X', 8)
    writer.sheets[hojas[2]].set_column('Y:Y', 15)
    writer.sheets[hojas[2]].set_column('Z:Z', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('AA:AA', 18, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('AB:AB', 8)

    ### 4. Rechazos

    writer.sheets[hojas[3]].set_column('A:A', 23)
    writer.sheets[hojas[3]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('E:E', 12)
    writer.sheets[hojas[3]].set_column('F:F', 20)
    writer.sheets[hojas[3]].set_column('G:G', 18)
    writer.sheets[hojas[3]].set_column('H:H', 15)
    writer.sheets[hojas[3]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('M:M', 21)
    writer.sheets[hojas[3]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('Q:Q', 60)
    writer.sheets[hojas[3]].set_column('R:R', 12)
    writer.sheets[hojas[3]].set_column('S:S', 18)
    writer.sheets[hojas[3]].set_column('T:T', 15)
    writer.sheets[hojas[3]].set_column('U:U', 15)
    writer.sheets[hojas[3]].set_column('V:V', 20)
    writer.sheets[hojas[3]].set_column('W:W', 16)
    writer.sheets[hojas[3]].set_column('X:X', 8)
    writer.sheets[hojas[3]].set_column('Y:Y', 18)
    writer.sheets[hojas[3]].set_column('Z:Z', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('AA:AA', 18, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('AB:AB', 8)

    ### 5. Ajustes

    writer.sheets[hojas[4]].set_column('A:A', 23)
    writer.sheets[hojas[4]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('E:E', 12)
    writer.sheets[hojas[4]].set_column('F:F', 20)
    writer.sheets[hojas[4]].set_column('G:G', 18)
    writer.sheets[hojas[4]].set_column('H:H', 15)
    writer.sheets[hojas[4]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('M:M', 24)
    writer.sheets[hojas[4]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('Q:Q', 60)
    writer.sheets[hojas[4]].set_column('R:R', 12)
    writer.sheets[hojas[4]].set_column('S:S', 18)
    writer.sheets[hojas[4]].set_column('T:T', 15)
    writer.sheets[hojas[4]].set_column('U:U', 15)
    writer.sheets[hojas[4]].set_column('V:V', 20)
    writer.sheets[hojas[4]].set_column('W:W', 16)
    writer.sheets[hojas[4]].set_column('X:X', 8)
    writer.sheets[hojas[4]].set_column('Y:Y', 15)
    writer.sheets[hojas[4]].set_column('Z:Z', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('AA:AA', 18, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('AB:AB', 18)

    ### 6. Devoluciones

    writer.sheets[hojas[5]].set_column('A:A', 16, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('B:B', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('C:C', 55, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('D:D', 12)
    writer.sheets[hojas[5]].set_column('E:E', 20)
    writer.sheets[hojas[5]].set_column('F:F', 18)
    writer.sheets[hojas[5]].set_column('G:G', 15)
    writer.sheets[hojas[5]].set_column('H:H', 21)
    writer.sheets[hojas[5]].set_column('I:I', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('J:J', 18, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('K:K', 20)
    writer.sheets[hojas[5]].set_column('L:L', 21, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('M:M', 37, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('N:N', 38, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('O:O', 34, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('P:P', 60)
    writer.sheets[hojas[5]].set_column('Q:Q', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('R:R', 14, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('S:S', 17, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('T:T', 19, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('U:U', 50, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('V:V', 50, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('W:W', 50, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('X:X', 75, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('Y:Y', 21)
    writer.sheets[hojas[5]].set_column('Z:Z', 10)
    writer.sheets[hojas[5]].set_column('AA:AA', 11)

    ## Filtros

    writer.sheets[hojas[0]].autofilter(0, 0, rep_facturas_seller.shape[0], rep_facturas_seller.shape[1]-1) 
    writer.sheets[hojas[1]].autofilter(0, 0, rep_rec_seller.shape[0], rep_rec_seller.shape[1]-1) 
    writer.sheets[hojas[2]].autofilter(0, 0, rep_liq_seller.shape[0], rep_liq_seller.shape[1]-1) 
    writer.sheets[hojas[3]].autofilter(0, 0, rep_rec_sin_seller.shape[0], rep_rec_sin_seller.shape[1]-1)
    writer.sheets[hojas[4]].autofilter(0, 0, rep_ajust_seller.shape[0], rep_ajust_seller.shape[1]-1)
    writer.sheets[hojas[5]].autofilter(0, 0, rep_devoluciones_seller.shape[0], rep_devoluciones_seller.shape[1]-1)

    ## Guardar Excel
    writer.save()

##### **5. Reporte consolidado de transacciones Falabella.com con Devoluciones**

In [ ]:
## Ruta
final_path = ruta_pagos + r'\\0. Reportes' 

## Listado de tablas a exportar
lista = [rep_transacciones]
hojas = ['data_transacciones']

## Nombre del archivo Excel
try:
    writer = pd.ExcelWriter(os.path.join(final_path, 'Reporte general de transacciones - Falabella.com' + '.xlsx'), engine='xlsxwriter')
except:
    writer = pd.ExcelWriter(os.path.join(final_path, 'Reporte general de transacciones - Falabella.com' + str(pd.to_datetime('today').date()) + '.xlsx'), engine='xlsxwriter')

## Exportar Excel
for j, df in enumerate(lista):
    ## Excel
    df.to_excel(writer, sheet_name=hojas[j], startrow=1, index=False, header=False)
    ## Define workbook and worksheet
    workbook = writer.book
    worksheet = writer.sheets[hojas[j]]
    ## Header format
    header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top', 'fg_color': '#D7E4BC'})
    ## Write headers with the defined format
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, header_format)
    ## Format Workbook
    workbook.formats[0].set_font_size(10) # Tamaño 10 letra
    ## Format Worksheet
    worksheet.set_zoom(70) # 70% zoom
    worksheet.freeze_panes(1, 0) # Freeze row 1
    worksheet.hide_gridlines(2) # Hide gridlines

    ## Widths

    ### 1. Data Transacciones

    writer.sheets[hojas[j]].set_column('A:A', 20)
    writer.sheets[hojas[j]].set_column('B:B', 16)
    writer.sheets[hojas[j]].set_column('C:C', 13)
    writer.sheets[hojas[j]].set_column('D:D', 51)
    writer.sheets[hojas[j]].set_column('E:E', 11)
    writer.sheets[hojas[j]].set_column('F:F', 20)
    writer.sheets[hojas[j]].set_column('G:G', 18)
    writer.sheets[hojas[j]].set_column('H:H', 15)
    writer.sheets[hojas[j]].set_column('I:I', 15)
    writer.sheets[hojas[j]].set_column('J:J', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('K:K', 10, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('L:L', 8, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('M:M', 7, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('N:N', 21, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('O:O', 18, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('P:P', 17, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('Q:Q', 14, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('R:R', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('S:S', 12)
    writer.sheets[hojas[j]].set_column('T:T', 18)
    writer.sheets[hojas[j]].set_column('U:U', 15)
    writer.sheets[hojas[j]].set_column('V:V', 11)
    writer.sheets[hojas[j]].set_column('W:W', 10)
    writer.sheets[hojas[j]].set_column('X:X', 17)
    writer.sheets[hojas[j]].set_column('Y:Y', 9)
    writer.sheets[hojas[j]].set_column('Z:Z', 14)
    writer.sheets[hojas[j]].set_column('AA:AA', 14)
    writer.sheets[hojas[j]].set_column('AB:AB', 12)
    writer.sheets[hojas[j]].set_column('AC:AC', 12)
    writer.sheets[hojas[j]].set_column('AD:AD', 12)
    writer.sheets[hojas[j]].set_column('AE:AE', 26)

## Filtros

writer.sheets[hojas[0]].autofilter(0, 0, rep_transacciones.shape[0], rep_transacciones.shape[1]-1) 

## Guardar Excel     
writer.save()

##### **6. Reporte semanales de liquidaciones Falabella.com para sellers**

In [ ]:
# 7. Reporte masivo semanal

if reporte_semanal_sellers!='no':

    max_week = facturas_seller.query("`Concepto`=='LIQUIDACIÓN' & `Documento`.notnull()")['Semana Recaudación'].max()
    final_path6 = ruta_pagos +  r'\\0. Reportes\\6. Reportes Semanales - Sellers\\2. Cortes\\' + str(max_week)

    #id_svl = ['5676']
    id_svl = facturas_seller['ID SELLER SVL'].astype(str).drop_duplicates().to_list()
    semanas = resumen_general['Semana Recaudación'].astype(str).drop_duplicates().to_list()

    for i in range(len(id_svl)):

        # 1. Definir

        ## Nombre del seller
        seller = facturas_seller[facturas_seller['ID SELLER SVL']==id_svl[i]]['Seller'].drop_duplicates().values[0]

        ## Calcular semana máxima reporte
        max_week = facturas_seller.query("`Concepto`=='LIQUIDACIÓN' & `Documento`.notnull()")['Semana Recaudación'].max()
        max_week_seller = template_rec[['Semana Recaudación']].sort_values(['Semana Recaudación'], ascending=False).drop_duplicates().max().values[0]

        ## Calcular fecha de compra máxima
        max_fecha_compra = rec_seller \
                                .query("`Fecha de Compra`.notnull()") \
                                .loc[:, ['Semana Recaudación', 'Fecha de Compra']].drop_duplicates() \
                                .groupby(['Semana Recaudación'], dropna=False) \
                                .agg({'Fecha de Compra': 'max'}).reset_index() \
                                .query("`Semana Recaudación` == @max_week_seller")['Fecha de Compra'].max()

        ## Filtros (se quita index sino no funciona con xlsxwriter)
        rep_resumen_seller = resumen_general[(resumen_general['ID SELLER SVL']==id_svl[i]) & (resumen_general['Semana Recaudación'] <= max_week_seller)].reset_index(drop=True).drop(['ID SELLER SVL', 'RUC seller', 'Seller'], axis=1)
        rep_facturas_seller = facturas_seller[facturas_seller['ID SELLER SVL']==id_svl[i]].reset_index(drop=True).drop(['Comentario'], axis=1)
        rep_rec_seller = rec_seller[(rec_seller['ID SELLER SVL']==id_svl[i]) & (rec_seller['Semana Recaudación']<=max_week_seller)].reset_index(drop=True)
        rep_liq_seller = liq_seller[(liq_seller['ID SELLER SVL']==id_svl[i]) & (liq_seller['Semana Recaudación']<=max_week_seller)].reset_index(drop=True)
        rep_rec_sin_seller = rec_sin_seller[(rec_sin_seller['ID SELLER SVL']==id_svl[i]) & (rec_sin_seller['Semana Recaudación']<=max_week_seller)].reset_index(drop=True)
        rep_ajust_seller = ajust_seller[(ajust_seller['ID SELLER SVL']==id_svl[i])]
        rep_devoluciones_seller = devoluciones_seller[(devoluciones_seller['ID SELLER SVL']==id_svl[i])]
        rep_trans_seller = trans_sellers[(trans_sellers['ID SELLER SVL'] == id_svl[i]) & (trans_sellers['Fecha de Compra'] <= max_fecha_compra)].reset_index(drop=True)

        ## Documentos
        try:
          abono = rep_facturas_seller.query("`Semana Recaudación` == @max_week & `Documento` =='FACTURA LIQUIDACIÓN'")['Número de documento'].values[0]
        except:
          abono = ''

        try:
          factura = rep_facturas_seller.query("`Semana Recaudación` == @max_week & `Documento` =='FACTURA COMISIÓN'")['Número de documento'].values[0]
        except:
          factura = ''
        
        try: 
          nc = rep_facturas_seller.query("`Semana Recaudación` == @max_week & `Documento` =='NOTA DE CRÉDITO COMISIÓN'")['Número de documento'].values[0]
        except:
          nc = ''

        # 2. Excel

        ## Listado de tablas a exportar
        lista = [rep_facturas_seller, rep_rec_seller, rep_liq_seller, rep_rec_sin_seller, rep_ajust_seller, rep_devoluciones_seller]
        hojas = ['Facturas', 'Recaudos', 'Liquidaciones', 'Rechazos', 'Ajustes', 'Devoluciones']

        ## Nombre del archivo Excel
        writer = pd.ExcelWriter(os.path.join(final_path6, ('FALABELLA MP - ' + str(id_svl[i]) + ' - ' + str(seller) + '.xlsx')), engine='xlsxwriter')

        ## Crear libro
        workbook = writer.book
        worksheet = workbook.add_worksheet('RESUMEN')

        ## Formatos

        title = workbook.add_format({'bold': True})
        data = workbook.add_format({'bold': True, 'align': 'left'})
        data_amount = workbook.add_format({'align': 'center'})
        line_item = workbook.add_format({'align': 'right'})
        total_item = workbook.add_format({'bold': True, 'align': 'center'})

        amount = workbook.add_format({
            'num_format': '_-[$S/-es-PE] * #,##0.00_-;-[$S/-es-PE] * #,##0.00_-;_-[$S/-es-PE] * "-"??_-;_-@_-',
            'align': 'center'})

        amount_total = workbook.add_format({
            'num_format': '_-[$S/-es-PE] * #,##0.00_-;-[$S/-es-PE] * #,##0.00_-;_-[$S/-es-PE] * "-"??_-;_-@_-',
            'align': 'center', 'bold': True})

        worksheet.set_column('S:S', None, None, {'hidden': True})

        ## Lista de rango de fechas

        for row_num, date in enumerate(semanas):
            worksheet.write(row_num, 18, date)

        worksheet.data_validation('D4', {'validate': 'list', 'source': '=$S$1:$S$90'})

        ## Escribir celdas y formulas

        worksheet.write('C1', 'Estado de Cuenta', title)

        worksheet.write('C3', 'RUC', data)
        worksheet.write_formula('D3', '=Facturas!$C$2', data_amount)

        worksheet.write('C4', 'Semana Recaudación (Periodo)', data)
        worksheet.write('D4', str(max_week), data_amount)

        ## Saldo inicial

        worksheet.write('B6', 'Saldo inicial', title)

        worksheet.write('C6', 'Balance negativo de estados de cuenta anteriores', line_item)
        worksheet.write_formula(
            'D6', '=IF(SUMIFS(Facturas!J:J,Facturas!$A:$A,"<"&RESUMEN!$D$4,Facturas!$E:$E,"LIQUIDACIÓN")-SUMIFS(Facturas!I:I,Facturas!$A:$A,"<"&RESUMEN!$D$4,Facturas!$E:$E,"LIQUIDACIÓN")>=-1,0,SUMIFS(Facturas!J:J,Facturas!$A:$A,"<"&RESUMEN!$D$4,Facturas!$E:$E,"LIQUIDACIÓN")-SUMIFS(Facturas!I:I,Facturas!$A:$A,"<"&RESUMEN!$D$4,Facturas!$E:$E,"LIQUIDACIÓN"))', amount_total)

        ## Órdenes

        worksheet.write('B8', 'Órdenes', title)

        worksheet.write('C8', 'Ingresos por ventas', line_item)
        worksheet.write_formula('D8', '=SUMIFS(Liquidaciones!U:U,Liquidaciones!$A:$A,RESUMEN!$D$4,Liquidaciones!$AB:$AB,"Ingreso")', amount)

        worksheet.write('C9', 'Reembolso de comisión', line_item)
        worksheet.write_formula('D9', '=-SUMIFS(Liquidaciones!$V:$V,Liquidaciones!$A:$A,RESUMEN!$D$4,Liquidaciones!$AB:$AB,"Egreso")', amount)

        worksheet.write('C10', 'Reembolso por Cofinanciamiento Logístico', line_item)
        worksheet.write_formula('D10', '=-SUMIFS(Liquidaciones!$W:$W,Liquidaciones!$A:$A,RESUMEN!$D$4,Liquidaciones!$AB:$AB,"Egreso")', amount)

        worksheet.write('C11', 'Subtotal', total_item)
        worksheet.write_formula('D11', '=D8+D9+D10', amount_total)

        ## Cobros

        worksheet.write('B13', 'Cobros', title)

        worksheet.write('C13', 'Órdenes devueltas', line_item)
        worksheet.write_formula('D13', '=SUMIFS(Liquidaciones!$U:$U,Liquidaciones!$A:$A,RESUMEN!$D$4,Liquidaciones!$AB:$AB,"Egreso")', amount)

        worksheet.write('C14', 'Comisión por venta', line_item)
        worksheet.write_formula('D14', '=-SUMIFS(Facturas!K:K,Facturas!E:E,"COMISIÓN",Facturas!A:A,RESUMEN!$D$4)', amount)

        worksheet.write('C15', 'Cofinanciamiento logístico', line_item)
        worksheet.write_formula('D15', '=-SUMIFS(Liquidaciones!$W:$W,Liquidaciones!$A:$A,RESUMEN!$D$4,Liquidaciones!$AB:$AB,"Ingreso")', amount)

        worksheet.write('C16', 'Regularizaciones (Otros Cobros)', line_item)
        worksheet.write_formula('D16', '=SUMIFS(Facturas!O:O,Facturas!E:E,"LIQUIDACIÓN",Facturas!A:A,RESUMEN!$D$4)', amount)

        worksheet.write('C17', 'Subtotal', total_item)
        worksheet.write_formula('D17', '=D13+D14+D15+D16', amount_total)

        ## Reembolsos

        worksheet.write('B19', 'Reembolsos', title)

        worksheet.write('C19', 'Devoluciones (Rechazos y Siniestros)', line_item)
        worksheet.write_formula('D19', '=SUMIFS(Facturas!M:M,Facturas!E:E,"DEVOLUCIONES",Facturas!A:A,RESUMEN!$D$4)', amount)

        worksheet.write('C20', 'Ajustes (Errores de cobros y precios)', line_item)
        worksheet.write_formula('D20', '=SUMIFS(Facturas!N:N,Facturas!E:E,"AJUSTES",Facturas!A:A,RESUMEN!$D$4)', amount)

        worksheet.write('C21', 'Subtotal', total_item)
        worksheet.write_formula('D21', '=D19+D20', amount_total)

        ## Saldo final

        worksheet.write('B23', 'Saldo final', title)

        worksheet.write('C23', 'Balance final', total_item)
        worksheet.write_formula('D23', '=D6+D11+D17+D21', amount_total)

        ## Pagos

        worksheet.write('B25', 'Pagos', title)

        worksheet.write('C25', 'Importe abonado (Liquidaciones)', line_item)
        worksheet.write_formula('D25', '=SUMIFS(Facturas!I:I,Facturas!E:E,"LIQUIDACIÓN",Facturas!A:A,RESUMEN!$D$4)', amount)

        worksheet.write('C26', 'Importe abonado (Reembolsos)', line_item)
        worksheet.write_formula('D26', '=SUMIFS(Facturas!M:M,Facturas!E:E,"DEVOLUCIONES",Facturas!A:A,RESUMEN!$D$4)', amount)

        worksheet.write('C27', 'Pago total', total_item)
        worksheet.write_formula('D27', '=D25+D26', amount_total)

        ## Format Worksheet
        worksheet.set_zoom(70)
        worksheet.hide_gridlines(2)
        worksheet.set_column('A:A', 1)
        worksheet.set_column('B:B', 11)
        worksheet.set_column('C:C', 47)
        worksheet.set_column('D:D', 19)

        ## --------- FIN CODIGO SAMUEL

        ## Exportar Excel
        for j, df in enumerate(lista):
            ## Excel
            df.to_excel(writer, sheet_name=hojas[j], startrow=1, index=False, header=False)
            ## Define workbook and worksheet
            workbook = writer.book
            worksheet = writer.sheets[hojas[j]]
            ## Header format
            header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top', 'fg_color': '#D7E4BC'})
            ## Write headers with the defined format
            for col_num, value in enumerate(df.columns.values):
                worksheet.write(0, col_num, value, header_format)
            ## Format Workbook
            workbook.formats[0].set_font_size(10) # Tamaño 10 letra
            ## Format Worksheet
            worksheet.set_zoom(70) # 70% zoom
            worksheet.freeze_panes(1, 0) # Freeze row 1
            worksheet.hide_gridlines(2) # Hide gridlines
            
        ## Widths

        ### 1. Facturas

        writer.sheets[hojas[0]].set_column('A:A', 23)
        writer.sheets[hojas[0]].set_column('B:B', 16)
        writer.sheets[hojas[0]].set_column('C:C', 13)
        writer.sheets[hojas[0]].set_column('D:D', 55)
        writer.sheets[hojas[0]].set_column('E:E', 14)
        writer.sheets[hojas[0]].set_column('F:F', 25, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('G:G', 36, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('H:H', 17)
        writer.sheets[hojas[0]].set_column('I:I', 11)
        writer.sheets[hojas[0]].set_column('J:J', 14, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('K:K', 12, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('L:L', 14, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('M:M', 16, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('N:N', 10, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('O:O', 17, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('P:P', 10)

        ### 2. Recaudos

        writer.sheets[hojas[1]].set_column('A:A', 23)
        writer.sheets[hojas[1]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('E:E', 12)
        writer.sheets[hojas[1]].set_column('F:F', 20)
        writer.sheets[hojas[1]].set_column('G:G', 18)
        writer.sheets[hojas[1]].set_column('H:H', 15)
        writer.sheets[hojas[1]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('M:M', 24)
        writer.sheets[hojas[1]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('Q:Q', 60)
        writer.sheets[hojas[1]].set_column('R:R', 12)
        writer.sheets[hojas[1]].set_column('S:S', 18)
        writer.sheets[hojas[1]].set_column('T:T', 15)
        writer.sheets[hojas[1]].set_column('U:U', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('V:V', 18, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('W:W', 8)

        ### 3. Liquidaciones

        writer.sheets[hojas[2]].set_column('A:A', 23)
        writer.sheets[hojas[2]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('E:E', 12)
        writer.sheets[hojas[2]].set_column('F:F', 20)
        writer.sheets[hojas[2]].set_column('G:G', 18)
        writer.sheets[hojas[2]].set_column('H:H', 15)
        writer.sheets[hojas[2]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('M:M', 24)
        writer.sheets[hojas[2]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('Q:Q', 60)
        writer.sheets[hojas[2]].set_column('R:R', 12)
        writer.sheets[hojas[2]].set_column('S:S', 18)
        writer.sheets[hojas[2]].set_column('T:T', 15)
        writer.sheets[hojas[2]].set_column('U:U', 15)
        writer.sheets[hojas[2]].set_column('V:V', 20)
        writer.sheets[hojas[2]].set_column('W:W', 16)
        writer.sheets[hojas[2]].set_column('X:X', 8)
        writer.sheets[hojas[2]].set_column('Y:Y', 15)
        writer.sheets[hojas[2]].set_column('Z:Z', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('AA:AA', 18, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('AB:AB', 8)

        ### 4. Rechazos

        writer.sheets[hojas[3]].set_column('A:A', 23)
        writer.sheets[hojas[3]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('E:E', 12)
        writer.sheets[hojas[3]].set_column('F:F', 20)
        writer.sheets[hojas[3]].set_column('G:G', 18)
        writer.sheets[hojas[3]].set_column('H:H', 15)
        writer.sheets[hojas[3]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('M:M', 21)
        writer.sheets[hojas[3]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('Q:Q', 60)
        writer.sheets[hojas[3]].set_column('R:R', 12)
        writer.sheets[hojas[3]].set_column('S:S', 18)
        writer.sheets[hojas[3]].set_column('T:T', 15)
        writer.sheets[hojas[3]].set_column('U:U', 15)
        writer.sheets[hojas[3]].set_column('V:V', 20)
        writer.sheets[hojas[3]].set_column('W:W', 16)
        writer.sheets[hojas[3]].set_column('X:X', 8)
        writer.sheets[hojas[3]].set_column('Y:Y', 18)
        writer.sheets[hojas[3]].set_column('Z:Z', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('AA:AA', 18, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('AB:AB', 8)

        ### 5. Ajustes

        writer.sheets[hojas[4]].set_column('A:A', 23)
        writer.sheets[hojas[4]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('E:E', 12)
        writer.sheets[hojas[4]].set_column('F:F', 20)
        writer.sheets[hojas[4]].set_column('G:G', 18)
        writer.sheets[hojas[4]].set_column('H:H', 15)
        writer.sheets[hojas[4]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('M:M', 24)
        writer.sheets[hojas[4]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('Q:Q', 60)
        writer.sheets[hojas[4]].set_column('R:R', 12)
        writer.sheets[hojas[4]].set_column('S:S', 18)
        writer.sheets[hojas[4]].set_column('T:T', 15)
        writer.sheets[hojas[4]].set_column('U:U', 15)
        writer.sheets[hojas[4]].set_column('V:V', 20)
        writer.sheets[hojas[4]].set_column('W:W', 16)
        writer.sheets[hojas[4]].set_column('X:X', 8)
        writer.sheets[hojas[4]].set_column('Y:Y', 15)
        writer.sheets[hojas[4]].set_column('Z:Z', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('AA:AA', 18, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('AB:AB', 18)

        ### 6. Devoluciones

        writer.sheets[hojas[5]].set_column('A:A', 16, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('B:B', 13, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('C:C', 55, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('D:D', 12)
        writer.sheets[hojas[5]].set_column('E:E', 20)
        writer.sheets[hojas[5]].set_column('F:F', 18)
        writer.sheets[hojas[5]].set_column('G:G', 15)
        writer.sheets[hojas[5]].set_column('H:H', 21)
        writer.sheets[hojas[5]].set_column('I:I', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('J:J', 18, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('K:K', 20)
        writer.sheets[hojas[5]].set_column('L:L', 21, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('M:M', 37, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('N:N', 38, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('O:O', 34, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('P:P', 60)
        writer.sheets[hojas[5]].set_column('Q:Q', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('R:R', 14, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('S:S', 17, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('T:T', 19, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('U:U', 50, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('V:V', 50, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('W:W', 50, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('X:X', 75, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('Y:Y', 21)
        writer.sheets[hojas[5]].set_column('Z:Z', 10)
        writer.sheets[hojas[5]].set_column('AA:AA', 11)

        ## Filtros

        ### 1. Facturas

        ##### Activar filtro
        writer.sheets[hojas[0]].autofilter(0, 0, rep_facturas_seller.shape[0], rep_facturas_seller.shape[1]-1) 
        writer.sheets[hojas[0]].filter_column_list('A', max_week)
        writer.sheets[hojas[0]].filter_column_list('E', ['LIQUIDACIÓN'])   

        ##### Esconder rows que no cumplen con el filtro
        for row, row_data in rep_facturas_seller.iterrows():
            region = row_data['Semana Recaudación']
            volume = row_data['Concepto']
            if not (region == max_week and volume == 'LIQUIDACIÓN'):
                writer.sheets[hojas[0]].set_row(row + 1, options={'hidden': True})

        ### 2. Recaudos

        ##### Activar filtro
        writer.sheets[hojas[1]].autofilter(0, 0, rep_rec_seller.shape[0], rep_rec_seller.shape[1]-1) 
        writer.sheets[hojas[1]].filter_column_list('A', max_week)
        
        ##### Esconder rows que no cumplen con el filtro
        for row, row_data in rep_rec_seller.iterrows():
            region = row_data['Semana Recaudación']
            if not (region == max_week):
                writer.sheets[hojas[1]].set_row(row + 1, options={'hidden': True})

        ### 3. Liquidaciones

        ##### Activar filtro
        writer.sheets[hojas[2]].autofilter(0, 0, rep_liq_seller.shape[0], rep_liq_seller.shape[1]-1) 
        writer.sheets[hojas[2]].filter_column_list('A', max_week)

        ##### Esconder rows que no cumplen con el filtro
        for row, row_data in rep_liq_seller.iterrows():
            region = row_data['Semana Recaudación']
            if not (region == max_week):
                writer.sheets[hojas[2]].set_row(row + 1, options={'hidden': True})

        ### 4. Rechazos

        ##### Activar filtro
        writer.sheets[hojas[3]].autofilter(0, 0, rep_rec_sin_seller.shape[0], rep_rec_sin_seller.shape[1]-1) 
        writer.sheets[hojas[3]].filter_column_list('A', max_week)

        ##### Esconder rows que no cumplen con el filtro
        for row, row_data in rep_rec_sin_seller.iterrows():
            region = row_data['Semana Recaudación']
            if not (region == max_week):
                writer.sheets[hojas[3]].set_row(row + 1, options={'hidden': True})

        ### 5. Ajustes

        ##### Activar filtro
        writer.sheets[hojas[4]].autofilter(0, 0, rep_ajust_seller.shape[0], rep_ajust_seller.shape[1]-1)
        writer.sheets[hojas[4]].filter_column_list('A', max_week)

        ##### Esconder rows que no cumplen con el filtro
        for row, row_data in rep_ajust_seller.iterrows():
            region = row_data['Semana Recaudación']
            if not (region == max_week):
                writer.sheets[hojas[4]].set_row(row + 1, options={'hidden': True})

        ### 6. Devoluciones

        writer.sheets[hojas[5]].autofilter(0, 0, rep_devoluciones_seller.shape[0], rep_devoluciones_seller.shape[1]-1)
        writer.sheets[hojas[5]].filter_column_list('H', ['ENTREGADO'])

        ##### Esconder rows que no cumplen con el filtro
        for row, row_data in rep_devoluciones_seller.iterrows():
            volume = row_data['Estado Egreso']
            if not (volume == 'ENTREGADO'):
                writer.sheets[hojas[5]].set_row(row + 1, options={'hidden': True})    

        ## Guardar Excel     
        writer.save()

        # 3. PDF

        ## 1. Resumen seller

        x = liq_seller[((liq_seller['ID SELLER SVL'] == id_svl[i])) & (liq_seller['Semana Recaudación']==max_week)]

        ## 2. Resumen de liquidaciones

        ### 1. Ingresos
        x_ingreso = x.query("Tipo=='Ingreso'") \
                    .loc[:, ['Folio Relacionado', 'Producto (SKU)', 'Liquidación']] \
                    .rename(columns={'Liquidación': 'Pago'}) \
                    .assign(**{'Pago': lambda x: x['Pago'].apply(lambda x: float("{:.2f}".format(x)))})

        ### 2. Egresos
        x_egreso = x.query("Tipo=='Egreso'") \
                    .loc[:, ['Folio Relacionado', 'Producto (SKU)', 'Liquidación']] \
                    .rename(columns={'Liquidación': 'Descuento'}) \
                    .assign(**{'Descuento': lambda x: x['Descuento'].apply(lambda x: float("{:.2f}".format(x)))})

        ### 3. Resumen
        x_resumen = pd.merge(x_ingreso, x_egreso, how='outer', on=['Folio Relacionado', 'Producto (SKU)']) \
                      .groupby(['Folio Relacionado', 'Producto (SKU)'], dropna=False) \
                      .agg({'Pago': 'sum', 'Descuento': 'sum'}).reset_index() \
                      .assign(**{'Liquidación': lambda x: x['Pago'] + x['Descuento']}) \
                      .sort_values(['Folio Relacionado']) \
                      .assign(**{'Liquidación': lambda x: x['Liquidación'].apply(lambda x: float("{:.2f}".format(x)))})

        ## 3. Resumen de facturas

        x_factura = x.query("Tipo=='Ingreso'") \
                    .loc[:, ['Folio Relacionado', 'Producto (SKU)', 'Importe Comisión', 'Cof. Logístico']] \
                    .assign(**{
                      'Importe Comisión': lambda x: x['Importe Comisión'].apply(lambda x: float("{:.2f}".format(x))),
                      'Cof. Logístico': lambda x: x['Cof. Logístico'].apply(lambda x: float("{:.2f}".format(x)))
                    })

        ## 4. Resumen de notas de crédito

        x_nc = x.query("Tipo=='Egreso'") \
                    .loc[:, ['Folio Relacionado', 'Producto (SKU)', 'Importe Comisión', 'Cof. Logístico']] \
                    .assign(**{
                      'Importe Comisión': lambda x: x['Importe Comisión'].apply(lambda x: float("{:.2f}".format(x))),
                      'Cof. Logístico': lambda x: x['Cof. Logístico'].apply(lambda x: float("{:.2f}".format(x)))
                    })

        ## 5. Crear PDF

        pdf = FPDF()

        ### 1. Liquidaciones

        #### Agregar hoja
        pdf.add_page()

        #### Title
        pdf.set_font('arial', 'B', 11)
        pdf.cell(50)
        pdf.cell(75, 10, 'Resumen de Liquidaciones - ' + str(abono), 0, 2, 'C')
        pdf.cell(50, 5, ' ', 0, 2, 'C')
        pdf.cell(-45)

        #### Headers
        pdf.set_font('arial', 'B', 9)
        columnNameList = list(x_resumen.columns)
        for header in columnNameList[:-1]:
          pdf.cell(35, 6, header, 1, 0, 'C')
        pdf.cell(35, 6, columnNameList[-1], 1, 2, 'C')

        #### Values
        pdf.cell(-140)
        pdf.set_font('arial', '', 8)
        for row in range(0, len(x_resumen)):
          for col_num, col_name in enumerate(columnNameList):
            if col_num != len(columnNameList)-1:
              pdf.cell(35, 4, str(x_resumen[col_name].iloc[row]), 1, 0, 'C')
            else:
              pdf.cell(35, 4, str(x_resumen[col_name].iloc[row]), 1, 2, 'R')  
              pdf.cell(-140)

        ### 2. Factura

        #### Agregar hoja
        pdf.add_page()

        #### Title
        pdf.set_font('arial', 'B', 11)
        pdf.cell(50)
        pdf.cell(75, 10, 'Resumen de importe de la factura - ' + str(factura), 0, 2, 'C')
        pdf.cell(50, 5, ' ', 0, 2, 'C')
        pdf.cell(-45)

        #### Headers
        pdf.set_font('arial', 'B', 9)
        col_factura = list(x_factura.columns)
        for header in col_factura[:-1]:
          pdf.cell(35, 6, header, 1, 0, 'C')
        pdf.cell(35, 6, col_factura[-1], 1, 2, 'C')

        #### Values
        pdf.cell(-105)
        pdf.set_font('arial', '', 8)
        for row in range(0, len(x_factura)):
          for col_num, col_name in enumerate(col_factura):
            if col_num != len(col_factura)-1:
              pdf.cell(35, 4, str(x_factura[col_name].iloc[row]), 1, 0, 'C')
            else:
              pdf.cell(35, 4, str(x_factura[col_name].iloc[row]), 1, 2, 'R')  
              pdf.cell(-105)

        ### 3. Nota de crédito

        #### Agregar hoja
        pdf.add_page()

        #### Title
        pdf.set_font('arial', 'B', 11)
        pdf.cell(50)
        pdf.cell(75, 10, 'Resumen de importe de la nota de crédito - ' + str(nc), 0, 2, 'C')
        pdf.cell(50, 5, ' ', 0, 2, 'C')
        pdf.cell(-45)

        #### Headers
        pdf.set_font('arial', 'B', 9)
        col_nc = list(x_nc.columns)
        for header in col_nc[:-1]:
          pdf.cell(35, 6, header, 1, 0, 'C')
        pdf.cell(35, 6, col_nc[-1], 1, 2, 'C')

        #### Values
        pdf.cell(-105)
        pdf.set_font('arial', '', 8)
        for row in range(0, len(x_nc)):
          for col_num, col_name in enumerate(col_nc):
            if col_num != len(col_nc)-1:
              pdf.cell(35, 4, str(x_nc[col_name].iloc[row]), 1, 0, 'C')
            else:
              pdf.cell(35, 4, str(x_nc[col_name].iloc[row]), 1, 2, 'R')
              pdf.cell(-105)

        #### Output
        pdf.output(os.path.join(final_path6, ('FALABELLA MP - ' + str(id_svl[i]) + ' - ' + str(seller) + '.pdf')), 'F') 

else:
    
    print("Para tener este reporte es necesario habilitar la opción de reportes sellers")